In [ ]:
# !pip uninstall torch -y

In [ ]:
# !pip install torch==2.2.1 -q

In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import json
import ast

In [ ]:
dataset = load_dataset("daily_dialog")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train = pd.DataFrame(dataset["train"]) #80% of the data set used to train the LLm
validation = pd.DataFrame(dataset["validation"]) #The correct output for the 20%
test = pd.DataFrame(dataset["test"]) #The output of the LLM

In [ ]:
train

,dialog,act,emotion
0,"[Say , Jim , how about going for a few beers a...","[3, 4, 2, 2, 2, 3, 4, 1, 3, 4]","[0, 0, 0, 0, 0, 0, 4, 4, 4, 4]"
1,"[Can you do push-ups ? , Of course I can . It...","[2, 1, 2, 2, 1, 1]","[0, 0, 6, 0, 0, 0]"
2,"[Can you study with the radio on ? , No , I l...","[2, 1, 2, 1, 1]","[0, 0, 0, 0, 0]"
3,"[Are you all right ? , I will be all right so...","[2, 1, 1, 1]","[0, 0, 0, 0]"
4,"[Hey John , nice skates . Are they new ? , Ye...","[2, 1, 2, 1, 1, 2, 1, 3, 4]","[0, 0, 0, 0, 0, 6, 0, 6, 0]"
...,...,...,...
11113,"[Hello , I bought a pen in your shop just befo...","[1, 1, 1, 2, 3, 2, 1, 4, 1]","[0, 4, 0, 0, 0, 0, 0, 0, 4]"
11114,"[Do you have any seats available ? , Yes . Th...","[2, 1, 2, 1, 3, 4]","[0, 0, 0, 0, 0, 4]"
11115,"[Uncle Ben , how did the Forbidden City get th...","[2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 3, 4]","[0, 0, 6, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0]"
11116,"[May I help you , sir ? , I want a pair of lo...","[2, 3, 4, 3]","[0, 0, 0, 0]"


In [ ]:
def format_GPT2_last_utterance(row):
    # Convert string representations into actual Python objects/lists
    dialog = row['dialog']
    emotions = row['emotion']

    formatted_dialog_GPT2 = ""
    for i, (utterance, emotion) in enumerate(zip(dialog, emotions)):

        speaker = "<Speaker_1>" if i % 2 == 0 else "<Speaker_2>"
        if i < len(dialog)-1:
            formatted_dialog_GPT2 += f"{speaker}: {utterance} | <Emotion>: {emotion} <|endofutterance|> "
        else:
            formatted_dialog_GPT2 += f"{speaker}: {utterance} | <Emotion>: "

    last_utterance_emotion = emotions[-1]

    return formatted_dialog_GPT2, last_utterance_emotion

In [ ]:
def format_GPT2_no_last_utterance(row):
    dialog = ast.literal_eval(row['dialog'])[:-1]  # Convert string to list and remove last utterance
    labels = row['classification']

    # Map classification to numeric labels
    label_map = {'No Derailment': 0, 'Positive Derailment': 1, 'Negative Derailment': 2}
    numeric_label = label_map[labels]

    formatted_dialog_GPT2 = ""
    for i, utterance in enumerate(dialog):
        speaker = "<Speaker_1>" if i % 2 == 0 else "<Speaker_2>"
        formatted_dialog_GPT2 += f"{speaker}: {utterance}<|endofutterance|> "

    return pd.Series([formatted_dialog_GPT2, numeric_label])

In [ ]:
## Formatting the training set

formatted_dialogs = []
labels = []
for _, row in train.iterrows():
    formatted_dialog, label = format_GPT2_last_utterance(row)
    formatted_dialogs.append(formatted_dialog)
    labels.append(label)

formatted_dialogs

["<Speaker_1>: Say , Jim , how about going for a few beers after dinner ?  | <Emotion>: 0 <|endofutterance|> <Speaker_2>:  You know that is tempting but is really not good for our fitness .  | <Emotion>: 0 <|endofutterance|> <Speaker_1>:  What do you mean ? It will help us to relax .  | <Emotion>: 0 <|endofutterance|> <Speaker_2>:  Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?  | <Emotion>: 0 <|endofutterance|> <Speaker_1>:  I guess you are right.But what shall we do ? I don't feel like sitting at home .  | <Emotion>: 0 <|endofutterance|> <Speaker_2>:  I suggest a walk over to the gym where we can play singsong and meet some of our friends .  | <Emotion>: 0 <|endofutterance|> <Speaker_1>:  That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .  | <Emotion>: 4 <|endofutterance|> <Speaker_2>:  Sounds great to me ! If they are willing , we could ask them to go dancing with us.

In [ ]:
## Formatting the validation set

formatted_dialogs_val = []
labels_val = []
for _, row in test.iterrows():
    formatted_dialog_val, label_val = format_GPT2_last_utterance(row)
    formatted_dialogs_val.append(formatted_dialog_val)
    labels_val.append(label_val)

In [ ]:
## Formatting the test set

formatted_dialogs_test = []
labels_test = []
for _, row in test.iterrows():
    formatted_dialog_test, label_test = format_GPT2_last_utterance(row)
    formatted_dialogs_test.append(formatted_dialog_test)
    labels_test.append(label_test)

we are now done formating the data.

In [ ]:
# pip install torch

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from tqdm import tqdm

# SETTING UP T4 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
torch.__version__

'2.2.1+cu121'

In [ ]:
# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',
                                          add_bos_token = True,
                                          add_eos_token = True)
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=7)
model.to(device)

# Define special tokens
special_tokens = {'bos_token': '<|startoftext|>','sep_token': '<|endofutterance|>', 'additional_special_tokens': ['<Speaker_1>', '<Speaker_2>', '<Emotion>']}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50262, 768)

In [ ]:
len(tokenizer)

50262

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|endoftext|>'

In [ ]:
tokenizer.sep_token

'<|endofutterance|>'

In [ ]:
tokenizer.bos_token

'<|startoftext|>'

In [ ]:
tokenizer.eos_token

'<|endoftext|>'

In [ ]:
# Tokenize train dataset
inputs = tokenizer(formatted_dialogs, padding=True, truncation=True, return_tensors="pt", max_length= 1024)
# Create TensorDataset
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels_tensor = torch.tensor(labels) #COME BACK TO THIS#
dataset = TensorDataset(input_ids, attention_mask, labels_tensor)

In [ ]:
from collections import Counter
label_count = Counter(labels)
print(label_count)

Counter({0: 7595, 4: 3031, 6: 159, 5: 141, 1: 130, 2: 47, 3: 15})


In [ ]:
input_ids


tensor([[50257, 50259,    25,  ..., 50256, 50256, 50256],
        [50257, 50259,    25,  ..., 50256, 50256, 50256],
        [50257, 50259,    25,  ..., 50256, 50256, 50256],
        ...,
        [50257, 50259,    25,  ..., 50256, 50256, 50256],
        [50257, 50259,    25,  ..., 50256, 50256, 50256],
        [50257, 50259,    25,  ..., 50256, 50256, 50256]])

In [ ]:
# inputs

In [ ]:
tokenizer.decode(input_ids[0])

"<|startoftext|> <Speaker_1> : Say, Jim, how about going for a few beers after dinner?  |  <Emotion> : 0 <|endofutterance|>  <Speaker_2> :  You know that is tempting but is really not good for our fitness.  |  <Emotion> : 0 <|endofutterance|>  <Speaker_1> :  What do you mean? It will help us to relax.  |  <Emotion> : 0 <|endofutterance|>  <Speaker_2> :  Do you really think so? I don't. It will just make us fat and act silly. Remember last time?  |  <Emotion> : 0 <|endofutterance|>  <Speaker_1> :  I guess you are right.But what shall we do? I don't feel like sitting at home.  |  <Emotion> : 0 <|endofutterance|>  <Speaker_2> :  I suggest a walk over to the gym where we can play singsong and meet some of our friends.  |  <Emotion> : 0 <|endofutterance|>  <Speaker_1> :  That's a good idea. I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them.  |  <Emotion> : 4 <|endofutterance|>  <Speaker_2> :  Sounds great to me! If they are willing, we could ask 

In [ ]:
#Tokenize the validation set
val_inputs = tokenizer(formatted_dialogs_val, padding=True, truncation=True, return_tensors="pt", max_length= 1024)
# Create TensorDataset for the validation set
val_input_ids = val_inputs['input_ids']
val_attention_mask = val_inputs['attention_mask']
val_labels_tensor = torch.tensor(labels_val) #COME BACK TO THIS#
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels_tensor)

In [ ]:
# val_inputs

In [ ]:
# Tokenize the test set
test_inputs = tokenizer(formatted_dialogs_test, padding=True, truncation=True, return_tensors="pt", max_length= 1024)
# Create a TensorDataset for the test set
test_input_ids = test_inputs['input_ids']
test_attention_mask = test_inputs['attention_mask']
test_labels_tensor = torch.tensor(labels_test) #COME BACK TO THIS#
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels_tensor)

In [ ]:
# test_inputs

In [ ]:
## Creating the data loaders
batch_size = 2
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

no idea after this

In [ ]:
import torch

In [ ]:
from torch.optim import AdamW

In [ ]:
# Defining the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
import copy

from sklearn.metrics import f1_score

# Training and Validation Loop

model.train()
num_epochs = 5
train_losses = []
val_losses = []
f1_scores = []
best_val_loss = float('inf')
patience = 4
epochs_no_improve = 0
early_stop = False

for epoch in tqdm(range(num_epochs), desc = "Epochs"):
    if early_stop:
        print("Early stopping")
        break

    print(f"Epoch {epoch+1} of {num_epochs}")

    #Training
    total_train_loss = 0
    for batch in tqdm(train_dataloader, desc='Training Progress', miniters=10):
        input_ids, attention_mask, batch_labels = batch
        input_ids, attention_mask, batch_labels = input_ids.to(device), attention_mask.to(device), batch_labels.to(device)
        model.config.pad_token_id = tokenizer.pad_token_id
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"Total training loss: {total_train_loss}")
    print(f"Training loss: {avg_train_loss}")

    #validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc='Validation Progress', miniters=10):
            input_ids, attention_masks, batch_labels = batch
            input_ids, attention_masks, batch_labels = input_ids.to(device), attention_masks.to(device), batch_labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=batch_labels)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)
            pred_classes = torch.argmax(probs, dim=1)

            val_predictions.extend(pred_classes.cpu().numpy())
            val_true_labels.extend(batch_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
    val_macro_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    f1_scores.append(val_macro_f1)
    print(f"Validation loss: {avg_val_loss}")
    print(f"Validation Macro F1 score: {val_macro_f1}")

    # Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            early_stop = True


    # Set the model back to training mode
    model.train()

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1 of 5



Training Progress: 100%|██████████| 5559/5559 [50:07<00:00,  1.85it/s]


Total training loss: 3429.8105737417936
Training loss: 0.616983373581902



Epochs:  20%|██        | 1/5 [51:04<3:24:18, 3064.55s/it]

...


Training Progress: 100%|██████████| 5559/5559 [50:11<00:00,  1.85it/s]


Total training loss: 1504.3922371580557
Training loss: 0.27062281654219383



Epochs:  80%|████████  | 4/5 [3:24:28<51:07, 3067.68s/it]  

Validation loss: 0.4429162908423459
Validation Macro F1 score: 0.5654415700495687
Epoch 5 of 5



Training Progress:  11%|█         | 600/5559 [05:24<44:47,  1.85it/s]